In [3]:
class PatchExtractor:
    def __init__(self, img, patch_size, stride):
        '''
        :param img: :py:class:`~PIL.Image.Image`
        :param patch_size: integer, size of the patch
        :param stride: integer, size of the stride
        '''
        self.img = img
        self.size = patch_size
        self.stride = stride

    def extract_patches(self):
        """
        extracts all patches from an image
        :returns: A list of :py:class:`~PIL.Image.Image` objects.
        """
        wp, hp = self.shape()
        return [self.extract_patch((w, h)) for h in range(hp) for w in range(wp)]

    def extract_patch(self, patch):
        """
        extracts a patch from an input image
        :param patch: a tuple
        :rtype: :py:class:`~PIL.Image.Image`
        :returns: An :py:class:`~PIL.Image.Image` object.
        """
        return self.img.crop((
            patch[0] * self.stride,  # left
            patch[1] * self.stride,  # up
            patch[0] * self.stride + self.size,  # right
            patch[1] * self.stride + self.size  # down
        ))

    def shape(self):
        wp = int((self.img.width - self.size) / self.stride + 1)
        hp = int((self.img.height - self.size) / self.stride + 1)
        return wp, hp


In [4]:
import glob
from PIL import Image

PATCH_SIZE = 256

# This is the folder you'll find after extracting the dataset.
train_folder = './data/images'
labels = glob.glob(train_folder +  '/*.tif')
labels.sort()

In [5]:
for item in labels:
  try:
    with Image.open(item) as img:
      extractor = PatchExtractor(img=img, patch_size=PATCH_SIZE, stride=256)
      patches = extractor.extract_patches()
      count = 0
      for p in patches:
        count += 1
#         print('./data/images/patches_i/' + str(count) + '_' + item.split('/')[-1][:-3] + 'png')
        p.save('./data/images/patches_i/' + str(count) + '_' + item.split('/')[-1][:-3] + 'png')
  except Exception as error:
    print('error with', item, error)

In [7]:
labels = glob.glob('./data/br_masks/*.png')
labels.sort()

In [8]:
for item in labels:
  try:
    with Image.open(item) as img:
      extractor = PatchExtractor(img=img, patch_size=PATCH_SIZE, stride=256)
      patches = extractor.extract_patches()
      count = 0
      for p in patches:
        count += 1
        # print('./train/' + value + '/' + str(count) + '_' + key.split('/')[-1])
        p.save('./data/br_masks/patches_m/' + str(count) + '_' + item.split('/')[-1])
  except Exception as error:
    print('error with', item, error)

In [9]:
images = [item for item in glob.glob('data/images/patches_i/*')]
masks = [item for item in glob.glob('data/br_masks/patches_m/*')]
images.sort()
masks.sort()

In [10]:
import sys
import os
import numpy as np
from keras.utils import Progbar
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input
# Function read train images and mask return as nump array
def read_train_data(IMG_WIDTH=256,IMG_HEIGHT=256,IMG_CHANNELS=3):
    X_train = np.zeros((len(images), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
    Y_train = np.zeros((len(masks), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
    print('Getting and resizing train images and masks ... ')
    sys.stdout.flush()
    if os.path.isfile("train_img.npy") and os.path.isfile("train_mask.npy"):
        print("Train file loaded from memory")
        X_train = np.load("train_img.npy")
        Y_train = np.load("train_mask.npy")
        return X_train,Y_train
    a = Progbar(len(images))
    for n, p in enumerate(zip(images, masks)):
        path = p[0]
        img = imread(path)[:,:,:IMG_CHANNELS]
        img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
        X_train[n] = img
        mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
        mask_ = imread(p[1])
        mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant', 
                                    preserve_range=True), axis=-1)
        mask = np.maximum(mask, mask_)
        Y_train[n] = mask
        a.update(n)
    np.save("train_img",X_train)
    np.save("train_mask",Y_train)
    return X_train,Y_train

Using TensorFlow backend.


In [12]:
train_img,train_mask = read_train_data()

Getting and resizing train images and masks ... 
12815/12816 [============================>.] - ETA: 0s

In [13]:
import os,time,cv2
import tensorflow as tf
import numpy as np
import tf_slim as slim

In [14]:
def ConvBlock(inputs, n_filters, kernel_size=[3, 3]):
	# Skip pointwise by setting num_outputs=Non
	net = slim.conv2d(inputs, n_filters, kernel_size=[1, 1], activation_fn=None)
	net = slim.batch_norm(net, fused=True)
	net = tf.nn.relu(net)
	return net

In [15]:
def DepthwiseSeparableConvBlock(inputs, n_filters, kernel_size=[3, 3]):

	# Skip pointwise by setting num_outputs=None
	net = slim.separable_convolution2d(inputs, num_outputs=None, depth_multiplier=1, kernel_size=[3, 3], activation_fn=None)

	net = slim.batch_norm(net, fused=True)
	net = tf.nn.relu(net)
	net = slim.conv2d(net, n_filters, kernel_size=[1, 1], activation_fn=None)
	net = slim.batch_norm(net, fused=True)
	net = tf.nn.relu(net)
	return net


In [16]:
def conv_transpose_block(inputs, n_filters, kernel_size=[3, 3]):

	net = slim.conv2d_transpose(inputs, n_filters, kernel_size=[3, 3], stride=[2, 2], activation_fn=None)
	net = tf.nn.relu(slim.batch_norm(net))
	return net

In [17]:

def build_ynet():


    
	# Downsampling path #
	
  inputs=Input((256,256,3))
  num_classes = 4
  net = ConvBlock(inputs, 64)
  net = DepthwiseSeparableConvBlock(net, 64)
  net = tf.nn.pool(net, [2, 2], stride=[2, 2], pooling_type='MAX')
  skip_1 = net

  net = DepthwiseSeparableConvBlock(net, 128)
  net = DepthwiseSeparableConvBlock(net, 128)
  net = tf.nn.pool(net, [2, 2], stride=[2, 2], pooling_type='MAX')
  skip_2 = net

  net = DepthwiseSeparableConvBlock(net, 256)
  net = DepthwiseSeparableConvBlock(net, 256)
  net = DepthwiseSeparableConvBlock(net, 256)
  net = tf.nn.pool(net, [2, 2], stride=[2, 2], pooling_type='MAX')
  skip_3 = net

  net = DepthwiseSeparableConvBlock(net, 512)
  net = DepthwiseSeparableConvBlock(net, 512)
  net = DepthwiseSeparableConvBlock(net, 512)
  net = tf.nn.pool(net, [2, 2], stride=[2, 2], pooling_type='MAX')
  skip_4 = net

  net = DepthwiseSeparableConvBlock(net, 512)
  net = DepthwiseSeparableConvBlock(net, 512)
  net = DepthwiseSeparableConvBlock(net, 512)
  net = tf.nn.pool(net, [2, 2], stride=[2, 2], pooling_type='MAX')


	
	# Upsampling path #
	
  net = conv_transpose_block(net, 512)
  net = DepthwiseSeparableConvBlock(net, 512)
  net = DepthwiseSeparableConvBlock(net, 512)
  net = DepthwiseSeparableConvBlock(net, 512)
  net = tf.add(net, skip_4)

  net = conv_transpose_block(net, 512)
  net = DepthwiseSeparableConvBlock(net, 512)
  net = DepthwiseSeparableConvBlock(net, 512)
  net = DepthwiseSeparableConvBlock(net, 256)
  net = tf.add(net, skip_3)

  net = conv_transpose_block(net, 256)
  net = DepthwiseSeparableConvBlock(net, 256)
  net = DepthwiseSeparableConvBlock(net, 256)
  net = DepthwiseSeparableConvBlock(net, 128)
  net = tf.add(net, skip_2)

  net = conv_transpose_block(net, 128)
  net = DepthwiseSeparableConvBlock(net, 128)
  net = DepthwiseSeparableConvBlock(net, 64)
  net = tf.add(net, skip_1)

  net = conv_transpose_block(net, 64)
  net = DepthwiseSeparableConvBlock(net, 64)
  net = DepthwiseSeparableConvBlock(net, 64)

	
	#      Softmax      #
	
  net = tf.nn.conv2d(net, num_classes, [1, 1], activation_fn=None, scope='logits')
  return net

In [20]:
model = build_ynet()
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_20 (Conv2D)              (None, 256, 256, 64) 1792        input_2[0][0]                    
__________________________________________________________________________________________________
dropout_10 (Dropout)            (None, 256, 256, 64) 0           conv2d_20[0][0]                  
__________________________________________________________________________________________________
conv2d_21 (Conv2D)              (None, 256, 256, 32) 18464       dropout_10[0][0]                 
____________________________________________________________________________________________

In [ ]:
model.fit(train_img,train_mask,batch_size=50,epochs=5)